# Cohort statistics

In [1]:
lib = r'/srv01/technion/shitay/Code/classifying_response_to_immunotherapy/'
import sys
sys.path.append(lib)
from utilities.package_importing import *

In [2]:
COHORT_PATH = r'/storage/md_keren/shitay/Data/droplet_seq/M97_M173/cohort/normalized/4.11.21/cohort_normalized_4.11.21_protein_coding_genes.pkl'

cohort = pickle.load(open(COHORT_PATH, 'rb'))

In [3]:
def extract_sample(sample_id):
    sample_path = join(SAMPLES_PATH, sample_id, f'{sample_id}.pkl')
    rna_sample = pickle.load(open(sample_path, 'rb'))
    return rna_sample

def extract_sample_no_wrapper_folder(sample_id):
    sample_path = join(SAMPLES_PATH, sample_id)
    rna_sample = pickle.load(open(sample_path, 'rb'))
    return rna_sample


    
def get_informative_df(rna_sample):
    interesting_properties = ['is_apoptosis', 
                              'is_immune',
                              'is_cancer',
                              'is_stromal',
                              'is_epithelial',
                              'cancer_immune_conflict', 
                              'is_doublet', 
                              'is_lymphoid',
                              'is_myeloid',
                              'should_be_removed',
                             'is_CelBender_empty']
    relevant_cells = {key:[cf for cf in rna_sample.cells_information if getattr(cf, key)] for key in interesting_properties}
    n_cells = len(relevant_cells)
    counter = {key1:{key2: 0 for key2 in interesting_properties} for key1 in interesting_properties}
    for prop1 in interesting_properties:
        for prop2 in interesting_properties:
            for cell_inf in relevant_cells[prop1]:
                counter[prop1][prop2] += int(getattr(cell_inf, prop2))
    df = pd.DataFrame(counter.values(), index=counter.keys())[counter.keys()]
    return  df

def check_cross_cell_types(sample_id):
    sample_path = join(SAMPLES_PATH, sample_id, f'{sample_id}.pkl')
    types = ['None', 'Immune_general', 'T cells', 'CD4 helper T cells', 'CD8 Cytotoxic T cells', 'Regulatory T cells',
             'Regulatory CD4 T cells', 'Regulatory CD8 T cells', 'Regulatory CD4_CD8 T cells', 'NKT cells', 'NK cells',
             'B cells', 'Activated T cells', 'Senescence T cells', 'Terminal effector', 'Exhausted T cells',
             'Stem_like T cells', 'Memory T cells', 'Memory CD4 T cells', 'Memory CD8 T cells',
             'Memory CD4_CD8 T cells', 'Macrophage_immature', 'Macrophage_mature', 'Monocyte_immature',
             'Monocyte_mature', 'cDCs_dendritic_cells', 'pDCs', 'myeloid cells_general_immature',
             'myeloid cells_general_mature', 'Neutrophils', 'Granolocytes']
    table = {k1: {k2: 0 for k2 in types} for k1 in types}
    data = pickle.load(open(sample_path, 'rb'))
    cls = [list(set(v)) for v in data.cells_information.getattr('cell_type_list')]
    cls = [v if len(v) else ['None'] for v in cls]

    for cell in cls:
        for t1 in cell:
            for t2 in cell:
                table[t1][t2] += 1
    df = pd.DataFrame(table.values(), index=table.keys())[table.keys()]
    return df

In [4]:

cross_check_df = None
df = get_informative_df(cohort)
cross_check_df

In [8]:
rna_sample = cohort
interesting_properties = ['is_apoptosis', 
                          'is_immune',
                          'is_cancer',
                          'is_stromal',
                          'is_epithelial',
                          'cancer_immune_conflict', 
                          'is_doublet', 
                          'is_lymphoid',
                          'is_myeloid',
                          'should_be_removed',
                         'is_CelBender_empty']
relevant_cells = {key:[cf for cf in rna_sample.cells_information if getattr(cf, key)] for key in interesting_properties}
n_cells = len(relevant_cells)
counter = {key1:{key2: 0 for key2 in interesting_properties} for key1 in interesting_properties}
for prop1 in interesting_properties:
    for prop2 in interesting_properties:
        for cell_inf in relevant_cells[prop1]:
            counter[prop1][prop2] += int(getattr(cell_inf, prop2))
df = pd.DataFrame(counter.values(), index=counter.keys())[counter.keys()]

In [9]:
df

,is_apoptosis,is_immune,is_cancer,is_stromal,is_epithelial,cancer_immune_conflict,is_doublet,is_lymphoid,is_myeloid,should_be_removed,is_CelBender_empty
is_apoptosis,0,0,0,0,0,0,0,0,0,0,0
is_immune,0,144577,0,0,0,0,1992,104144,25211,0,30534
is_cancer,0,0,157733,0,0,0,4678,0,0,0,9573
is_stromal,0,0,0,33897,4930,0,25,0,0,0,6208
is_epithelial,0,0,0,4930,4930,0,1,0,0,0,909
cancer_immune_conflict,0,0,0,0,0,0,0,0,0,0,0
is_doublet,0,1992,4678,25,1,0,6695,95,1746,0,0
is_lymphoid,0,104144,0,0,0,0,95,104144,0,0,22570
is_myeloid,0,25211,0,0,0,0,1746,0,25211,0,4202
should_be_removed,0,0,0,0,0,0,0,0,0,0,0
